In [14]:
#initialising the libraries

from ultralytics import YOLO
import os

In [15]:
#defining all the paths

# Main directories
BASE_DIR = "/Users/surisettivamsikrishna/Downloads/Vamsi Pc/CODES/Mark2/River Ganges"
DATASET_DIR = os.path.join(BASE_DIR, "Dataset")

# Dataset YAML path
DATASET_YAML = DATASET_YAML = os.path.join(BASE_DIR, "dataset.yaml")


# Pre-trained YOLO model weights
MODEL_PATH = os.path.join(BASE_DIR, "Core Model/yolov8s.pt")




In [16]:
# Create dataset.yaml
yaml_content = f"""
path: {DATASET_DIR}
train: train/images
val: valid/images

nc: 2  # Number of classes
names: ['plastic', 'river']  # Class names
"""

# Save the YAML file
with open(DATASET_YAML, "w") as f:
    f.write(yaml_content)
print("Dataset YAML created successfully!")


Dataset YAML created successfully!


In [17]:
# Initialize the YOLO model with pre-trained weights
model = YOLO(MODEL_PATH)

/Users/surisettivamsikrishna/Downloads/Vamsi Pc/CODES/Mark2/River Ganges/Core Model/yolov8s.pt

In [18]:

# Train the model
model.train(
    data="/Users/surisettivamsikrishna/Downloads/Vamsi Pc/CODES/Mark2/River Ganges/Dataset.yaml",  # Path to YAML
    epochs=50,      # Number of epochs (adjust as needed)
    imgsz=640,      # Image size
    batch=16,       # Batch size
    workers=4,      # Number of data-loading workers
    name="river_plastic_detection"  # Experiment name
)


New https://pypi.org/project/ultralytics/8.3.55 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.54 🚀 Python-3.12.0 torch-2.5.1 CPU (Apple M1)
engine/trainer: task=detect, mode=train, model=/Users/surisettivamsikrishna/Downloads/Vamsi Pc/CODES/Mark2/River Ganges/Core Model/yolov8s.pt, data=/Users/surisettivamsikrishna/Downloads/Vamsi Pc/CODES/Mark2/River Ganges/Dataset.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=4, project=None, name=river_plastic_detection5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, s

train: Scanning /Users/surisettivamsikrishna/Downloads/Vamsi Pc/CODES/Mark2/River Ganges/Dataset/train/labels.cache... 1026 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1026/1026 [00:00<?, ?it/s]

train: WARNING ⚠️ /Users/surisettivamsikrishna/Downloads/Vamsi Pc/CODES/Mark2/River Ganges/Dataset/train/images/Aerial_Location_1_104_jpg.rf.74cc6c2ebb281ce4f993bfc73f302d2c.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /Users/surisettivamsikrishna/Downloads/Vamsi Pc/CODES/Mark2/River Ganges/Dataset/train/images/Aerial_Location_1_104_jpg.rf.ee028aea28f3bbe7aa16838224499521.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /Users/surisettivamsikrishna/Downloads/Vamsi Pc/CODES/Mark2/River Ganges/Dataset/train/images/Aerial_Location_1_164_jpg.rf.7b8c09447b9d7cd7cca62a2ab8212144.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /Users/surisettivamsikrishna/Downloads/Vamsi Pc/CODES/Mark2/River Ganges/Dataset/train/images/Aerial_Location_1_218_jpg.rf.707f4c5cbe05d9517ecce46155fab395.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /Users/surisettivamsikrishna/Downloads/Vamsi Pc/CODES/Mark2/River Ganges/Dataset/train/images/Aerial_Location_1_46_jpg.rf.29cfccb7e9929931857a7e48c1b073bf.jpg


val: Scanning /Users/surisettivamsikrishna/Downloads/Vamsi Pc/CODES/Mark2/River Ganges/Dataset/valid/labels.cache... 43 images, 0 backgrounds, 0 corrupt: 100%|██████████| 43/43 [00:00<?, ?it/s]

Plotting labels to runs/detect/river_plastic_detection5/labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/river_plastic_detection5
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/65 [00:09<?, ?it/s]


IndexError: index 2 is out of bounds for dimension 1 with size 2

/Users/surisettivamsikrishna/Downloads/Vamsi Pc/CODES/Mark2/River Ganges/Dataset/train/labels

In [47]:
import os

train_images = "/Users/surisettivamsikrishna/Downloads/Vamsi Pc/CODES/Mark2/River Ganges/Dataset/train/images"
train_labels = "/Users/surisettivamsikrishna/Downloads/Vamsi Pc/CODES/Mark2/River Ganges/Dataset/train/labels"

# Ensure matching image and label files
image_files = {os.path.splitext(img)[0] for img in os.listdir(train_images) if img.endswith('.jpg')}
label_files = {os.path.splitext(lbl)[0] for lbl in os.listdir(train_labels) if lbl.endswith('.txt')}

# Find unmatched files
unmatched_images = image_files - label_files
unmatched_labels = label_files - image_files

# Remove unmatched files
for img in unmatched_images:
    try:
        os.remove(os.path.join(train_images, f"{img}.jpg"))
    except FileNotFoundError:
        print(f"File {img}.jpg not found, skipping...")

for lbl in unmatched_labels:
    try:
        os.remove(os.path.join(train_labels, f"{lbl}.txt"))
    except FileNotFoundError:
        print(f"File {lbl}.txt not found, skipping...")

print("Unmatched files removed!")


Unmatched files removed!
